In [0]:
import dataiku
import pandas as pd

In [0]:
client = dataiku.api_client()
users = client.list_users()
user_data = []

In [0]:
for i in users:
    myuser = client.get_user(i['login'])
    
    user_data.append({
        "Display name": i.get('displayName', ''),
        "Last successful login": str(myuser.get_activity().last_successful_login),
        "Last session activity": str(myuser.get_activity().last_session_activity),
        'Profile':i.get('userProfile',''),
        'Trial Status':i.get('trialStatus',''),
        "Groups": ", ".join(i.get('groups', [])),
        "User creation date": str(myuser.get_settings().creation_date),
        "Type": i.get('sourceType', ''),
        "Email": i.get('email', 'N/A'),
        "Login name": i.get('login', ''),
    })

In [0]:
df = pd.DataFrame(user_data).sort_values(by=["Last session activity", "Last successful login"], ascending=False)

In [0]:
df["Last successful login"] = pd.to_datetime(df["Last successful login"], errors='coerce').dt.tz_localize(None)
df["Last session activity"] = pd.to_datetime(df["Last session activity"], errors='coerce').dt.tz_localize(None)
now = pd.Timestamp.now().tz_localize(None)
df["Most recent activity"] = df[["Last successful login", "Last session activity"]].max(axis=1)
df["Time since last activity"] = now - df["Most recent activity"]

In [0]:
def format_timedelta(td):
    if pd.isna(td):
        return "N/A"
    seconds = int(td.total_seconds())
    days, seconds = divmod(seconds, 86400)
    hours, seconds = divmod(seconds, 3600)
    minutes, seconds = divmod(seconds, 60)

    parts = []
    if days > 0:
        parts.append(f"{days} day{'s' if days != 1 else ''}")
    if hours > 0:
        parts.append(f"{hours} hour{'s' if hours != 1 else ''}")
    if minutes > 0:
        parts.append(f"{minutes} minute{'s' if minutes != 1 else ''}")
    if not parts:
        parts.append(f"{seconds} second{'s' if seconds != 1 else ''}")
    
    return ", ".join(parts)

df["Time since last activity"] = df["Time since last activity"].apply(format_timedelta)

In [0]:
df = df[~df['Login name'].str.endswith('@dataiku.com', na=False)]

In [0]:
priority_columns = ["Display name", "Time since last activity"]
other_columns = [col for col in df.columns if col not in priority_columns]
df = df[priority_columns + other_columns]
df

In [0]:
# Recipe outputs
# users_profiles_trialstatus_groups = dataiku.Dataset("Users_Profiles_TrialStatus_Groups")
# users_profiles_trialstatus_groups.write_with_schema(df)